# Eat Safe, Love

## Notebook Set Up

In [ ]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}
hygiene20 = establishments.find(query)

# Use count_documents to display the number of documents in the result
count_results = establishments.count_documents(query)
print(f'Number of establishments with a hygiene score of 20: {count_results}'
      
# Display the first document in the results using pprint
print("Example of establishments with a hygiene score of 20:")
pprint(hygiene20[0])

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene20_df =pd.DataFrame(list(hygiene20))

# Display the number of rows in the DataFrame
print(f'Number of rows in hygiene20_df: {len(hygiene20_df)}')

# Display the first 10 rows of the DataFrame
hygiene20_df.head(10)

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
london_query = {'LocalAuthorityName': {'$regex': "London"},
         'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
count_results = establishments.count_documents(london_query)
print(f'Establishments with London as the Local Authority and has a RatingValue greater than or equal to 4: {count_results}')
print()

# Display the first document in the results using pprint
london_results = establishments.find(london_query)
pprint(list(london_results)[0])

In [ ]:
# Convert the result to a Pandas DataFrame
london_results_df = pd.DataFrame(list(london_results)

# Display the number of rows in the DataFrame
print(f'Number of rows in london_results_df: {len(london_results_df)}')

# Display the first 10 rows of the DataFrame
print(london_results_df.head(10))

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

rating_latlon_query = {'RatingValue': {'$eq': 5},
                       'geocode.longitude': {'$gt': (longitude - degree_search), '$lt': (longitude + degree_search)},
                       'geocode.latitude': {'$gt': (latitude - degree_search), '$lt': (latitude + degree_search)}}
sort = [('scores.Hygiene', -1)]
limit = 5


# Print the results
rating_latlon_result = list(establishments.find(rating_latlon_query).sort(sort).limit(limit))  # Fixed query name
pprint(rating_latlon_result)

In [ ]:
# Convert result to Pandas DataFrame
rating_latlon_result_df = pd.DataFrame(rating_latlon_result)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Define the aggregation pipeline
match_query = {'$match': {'scores.Hygiene': {'$eq': 0}}}
group_query = {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}}
sort_values = {'$sort': {'count': -1}}

pipeline = [match_query, group_query, sort_values]
results = list(establishments.aggregate(pipeline))  # Fixed

# Print the number of documents in the result
print("Rows in DataFrame: ", len(results))

# Print the first 10 results
pprint(results[0:10])

In [ ]:
# Convert the result to a Pandas DataFrame
results_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(results_df))

# Display the first 10 rows of the DataFrame
print(results_df.head(10))